# Configurações Iniciais

In [0]:
%pip install mlflow>=3.0 --upgrade
%pip install lightgbm xgboost catboost scikit-plot imbalanced-learn synapseml
%pip install lightgbm
dbutils.library.restartPython()

In [0]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import (
    roc_auc_score, average_precision_score, 
    precision_score, recall_score, f1_score, 
    precision_recall_curve, classification_report,
    confusion_matrix
)
from sklearn.model_selection import cross_val_score, StratifiedKFold
import mlflow.pyfunc
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature
import warnings
warnings.filterwarnings('ignore')

# Stacking Essemble

In [0]:
X_train_scaled = spark.read.table("my_catalog.default.creditcard_x_train_scaled_tcc").toPandas()
X_test_scaled = spark.read.table("my_catalog.default.creditcard_x_test_scaled_tcc").toPandas()

y_train = spark.read.table("my_catalog.default.creditcard_train_pdd_tcc").toPandas()["Class"]
y_test = spark.read.table("my_catalog.default.creditcard_test_pd_tcc").toPandas()["Class"]

In [0]:
model_uris = {
    "XGBoost": "models:/workspace.default.creditcard-fraud-xgboost-optimized/1",
    "RandomForest": "models:/workspace.default.creditcard-fraud-randomforest-smote/1",
    "CatBoost": "models:/workspace.default.creditcard-fraud-catboost-optimized/1",
    "LightGBM": "models:/workspace.default.creditcard-fraud-lightgbm-smote/1",
    "MLP_Classifier": "models:/workspace.default.creditcard-fraud-mlp_classifier-optimized/1"
}

base_models = {}
for model_name, model_uri in model_uris.items():
    model = mlflow.pyfunc.load_model(model_uri)
    base_models[model_name] = model
    print(f"Modelo {model_name} carregado com sucesso.")

In [0]:
#import pandas as pd

train_base_predictions = {}
for model_name, model in base_models.items():
    if hasattr(model, "predict_proba"):
        train_base_predictions[model_name] = model.predict_proba(X_train_scaled)[:, 1]
    else:
        train_base_predictions[model_name] = model.predict(X_train_scaled)

train_meta_features = pd.DataFrame(train_base_predictions)

train_meta_features['Class'] = y_train

test_base_predictions = {}
for model_name, model in base_models.items():
    if hasattr(model, "predict_proba"):
        test_base_predictions[model_name] = model.predict_proba(X_test_scaled)[:, 1]
    else:
        test_base_predictions[model_name] = model.predict(X_test_scaled)

test_meta_features = pd.DataFrame(test_base_predictions)

In [0]:
meta_learner = LogisticRegression(class_weight='balanced', random_state=42, max_iter=1000)
meta_learner.fit(train_meta_features.drop('Class', axis=1), train_meta_features['Class'])

In [0]:
y_pred_meta = meta_learner.predict(test_meta_features)
y_prob_meta = meta_learner.predict_proba(test_meta_features)[:, 1]

roc_auc_meta = roc_auc_score(y_test, y_prob_meta)
avg_precision_meta = average_precision_score(y_test, y_prob_meta)
precision_meta = precision_score(y_test, y_pred_meta, zero_division=0)
recall_meta = recall_score(y_test, y_pred_meta)
f1_meta = f1_score(y_test, y_pred_meta)

print(f"Stacking Ensemble - ROC-AUC: {roc_auc_meta:.4f}, Avg Precision: {avg_precision_meta:.4f}, Precision: {precision_meta:.4f}, Recall: {recall_meta:.4f}, F1-Score: {f1_meta:.4f}")

In [0]:

print("\nAjustes finos na regressão logística:\n")

class_weights = [
    'balanced',
    {0: 1, 1: 5},
    {0: 1, 1: 8},
    {0: 1, 1: 10},
    {0: 1, 1: 15},
    {0: 1, 1: 20}
]

C_values = [0.001, 0.01, 0.1, 1, 10, 100]

lr_results = []

for weight in class_weights:
    for C in C_values:
        try:
            lr = LogisticRegression(
                C=C,
                class_weight=weight,
                random_state=42,
                max_iter=1000,
                solver='liblinear'
            )
            
            lr.fit(train_meta_features.drop('Class', axis=1), train_meta_features['Class'])
            
            y_pred_lr = lr.predict(test_meta_features)
            y_prob_lr = lr.predict_proba(test_meta_features)[:, 1]
            
            roc_auc = roc_auc_score(y_test, y_prob_lr)
            avg_precision = average_precision_score(y_test, y_prob_lr)
            precision = precision_score(y_test, y_pred_lr, zero_division=0)
            recall = recall_score(y_test, y_pred_lr)
            f1 = f1_score(y_test, y_pred_lr)
            
            lr_results.append({
                'C': C,
                'class_weight': str(weight),
                'ROC-AUC': roc_auc,
                'Avg Precision': avg_precision,
                'Precision': precision,
                'Recall': recall,
                'F1-Score': f1
            })
            
        except Exception as e:
            print(f"Erro com C={C}, weight={weight}: {e}")

lr_results_df = pd.DataFrame(lr_results)
print("\nMelhores configurações de Logistic Regression:")
print(lr_results_df.sort_values('F1-Score', ascending=False).head(10).to_string(index=False))

In [0]:
print("Melhor configuração:")
print("C: 0.001, class_weight: {0: 1, 1: 8}")
print("ROC-AUC: 0.9126, Avg Precision: 0.7877, Precision: 0.9268, Recall: 0.7755, F1-Score: 0.8444")

| Modelo        | ROC-AUC | Avg Precision | Precision | Recall | F1-Score |
|----------------|--------|---------------|-----------|--------|----------|
| Stacking Ensemble          | 0.9126 | 0.7877        | 0.9268    | 0.7755 | 0.8444   |

In [0]:
final_meta_learner = LogisticRegression(
    C=0.001,
    class_weight={0: 1, 1: 8},
    random_state=42,
    max_iter=1000,
    solver='liblinear'
)

final_meta_learner.fit(train_meta_features.drop('Class', axis=1), train_meta_features['Class'])

y_pred_final = final_meta_learner.predict(test_meta_features)
y_prob_final = final_meta_learner.predict_proba(test_meta_features)[:, 1]

roc_auc_final = roc_auc_score(y_test, y_prob_final)
avg_precision_final = average_precision_score(y_test, y_prob_final)
precision_final = precision_score(y_test, y_pred_final, zero_division=0)
recall_final = recall_score(y_test, y_pred_final)
f1_final = f1_score(y_test, y_pred_final)

print(f"Modelo final otimizado:")
print(f"F1-Score: {f1_final:.4f} (Original: 0.8432)")
print(f"Precision: {precision_final:.4f} (Original: 0.8966)")
print(f"Recall: {recall_final:.4f} (Original: 0.7959)")
print(f"Avg precision: {avg_precision_final:.4f}")


cm_final = confusion_matrix(y_test, y_pred_final)
print(f"\nMatriz de Confusão Final:")
print(cm_final)

## Salvar no MLFLow

In [0]:
mlflow.set_experiment("/Users/2106144@aluno.univesp.br/creditcard-fraud-detection")

In [0]:
sample_input = pd.DataFrame({
    name: [0.5] * len(base_models) for name in base_models.keys()
})
sample_output = final_meta_learner.predict_proba(sample_input)[:, 1]
signature = infer_signature(sample_input, sample_output)

with mlflow.start_run(run_name="stacking_ensemble_complete") as run_complete:
    class StackingEnsemble(mlflow.pyfunc.PythonModel):
        def __init__(self, base_models, meta_learner):
            self.base_models = base_models
            self.meta_learner = meta_learner

        def predict(
            self, 
            context, 
            model_input: pd.DataFrame
        ) -> np.ndarray:
            meta_features = {}
            for name, model in self.base_models.items():
                if hasattr(model, "predict_proba"):
                    meta_features[name] = model.predict_proba(model_input)[:, 1]
                else:
                    meta_features[name] = model.predict(model_input)
            meta_df = pd.DataFrame(meta_features)
            return self.meta_learner.predict_proba(meta_df)[:, 1]

    ensemble_model = StackingEnsemble(base_models, final_meta_learner)

    mlflow.pyfunc.log_model(
        name="complete_stacking_ensemble",
        python_model=ensemble_model,
        registered_model_name="workspace.default.creditcard-fraud-complete-stacking-ensemble",
        signature=signature
    )

    mlflow.log_metrics({
        "roc_auc": roc_auc_final,
        "avg_precision": avg_precision_final,
        "precision": precision_final,
        "recall": recall_final,
        "f1_score": f1_final
    })

    print(f"\nEnsemble completo salvo com sucesso!")
    print(f"Run ID: {run_complete.info.run_id}")

print("\n Modelos Registrados\n")
from mlflow.tracking import MlflowClient
client = MlflowClient()

In [0]:
model_name = "workspace.default.creditcard-fraud-complete-stacking-ensemble"
try:
    versions = client.search_model_versions(f"name='{model_name}'")
    print(f"\n{model_name}: \n")
    for v in versions:
        print(f" Versão {v.version} - {v.status} - {v.run_id}\n")
except:
    print(f"\n Modelo {model_name} não encontrado")

## Desempenho Preditivo

In [0]:
metrics = {
    "roc_auc": roc_auc_final,
    "avg_precision": avg_precision_final,
    "precision": precision_final,
    "recall": recall_final,
    "f1_score": f1_final
}

normalized_metrics = {k: (v if v is not None else 0) for k, v in metrics.items()}

weights = {
    "roc_auc": 0.2,
    "avg_precision": 0.25,
    "precision": 0.15,
    "recall": 0.2,
    "f1_score": 0.2
}

stacking_score = sum(normalized_metrics[k] * weights[k] for k in metrics.keys())

print("Análise das métricas do modelo Stacking:")
for k, v in metrics.items():
    print(f"{k}: {v:.4f}")

print(f"\nScore combinado para problemas desbalanceados: {stacking_score:.4f}")
print("Use este score para comparar modelos: quanto maior, melhor o desempenho geral considerando desbalanceamento.")

**Resultado de Desempenho Preditivo do Modelo Stacking Ensemble**

| Métrica         | roc_auc | avg_precision | precision | recall | f1_score | Score combinado |
|-----------------|---------|---------------|-----------|--------|----------|-----------------|
| Valor           | 0.9126  | 0.7877        | 0.9268    | 0.7755 | 0.8444   | 0.8425          |

## Eficiência Computacional

In [0]:
import time
import psutil
import os

n_runs = 10
inference_times = []
memory_usages = []
cpu_percents = []
num_threads_list = []

for _ in range(n_runs):
    start_time = time.time()
    y_pred_final = final_meta_learner.predict(test_meta_features)
    inference_time = time.time() - start_time
    inference_times.append(inference_time)

    process = psutil.Process(os.getpid())
    memory_usages.append(process.memory_info().rss / (1024 * 1024))
    cpu_percents.append(psutil.cpu_percent(interval=0.1))
    num_threads_list.append(process.num_threads())

ensemble_efficiency = {
    "model": "StackingEnsemble",
    "inference_time_sec": np.mean(inference_times),
    "memory_usage_mb": np.mean(memory_usages),
    "cpu_percent": np.mean(cpu_percents),
    "num_threads": np.mean(num_threads_list)
}

mlflow.log_metric("ensemble_inference_time_sec", ensemble_efficiency["inference_time_sec"])
mlflow.log_metric("ensemble_memory_usage_mb", ensemble_efficiency["memory_usage_mb"])
mlflow.log_metric("ensemble_cpu_percent", ensemble_efficiency["cpu_percent"])
mlflow.log_metric("ensemble_num_threads", ensemble_efficiency["num_threads"])

ensemble_efficiency_df = pd.DataFrame([ensemble_efficiency])
display(ensemble_efficiency_df)

In [0]:
max_inference_time = 5.0  # segundos
max_memory_usage = 1024.0  # MB
max_cpu_percent = 100.0
max_num_threads = 16.0

efficiency_metrics = {
    "inference_time_sec": ensemble_efficiency["inference_time_sec"],
    "memory_usage_mb": ensemble_efficiency["memory_usage_mb"],
    "cpu_percent": ensemble_efficiency["cpu_percent"],
    "num_threads": ensemble_efficiency["num_threads"]
}

efficiency_score = (
    (1 - min(efficiency_metrics["inference_time_sec"] / max_inference_time, 1)) * 0.4 +
    (1 - min(efficiency_metrics["memory_usage_mb"] / max_memory_usage, 1)) * 0.4 +
    (1 - min(efficiency_metrics["cpu_percent"] / max_cpu_percent, 1)) * 0.1 +
    (1 - min(efficiency_metrics["num_threads"] / max_num_threads, 1)) * 0.1
)

overall_score = stacking_score * 0.7 + efficiency_score * 0.3

print(f"Score de eficiência computacional: {efficiency_score:.4f}")
print(f"Score combinado (trade-off): {overall_score:.4f}")

mlflow.log_metric("efficiency_score", efficiency_score)
mlflow.log_metric("overall_score", overall_score)

In [0]:
from pyspark.sql import Row

metrics_row = Row(
    model="StackingEnsemble",
    roc_auc=float(roc_auc_final),
    avg_precision=float(avg_precision_final),
    precision=float(precision_final),
    recall=float(recall_final),
    f1_score=float(f1_final),
    weighted_score=float(stacking_score),
    inference_time_sec=float(ensemble_efficiency["inference_time_sec"]),
    memory_usage_mb=float(ensemble_efficiency["memory_usage_mb"]),
    cpu_percent=float(ensemble_efficiency["cpu_percent"]),
    num_threads=float(ensemble_efficiency["num_threads"]),
    efficiency_score=float(efficiency_score),
    overall_score=float(overall_score)
)

metrics_spark_df = spark.createDataFrame([metrics_row])

metrics_spark_df.write.mode("append").saveAsTable("my_catalog.default.stacking_final_metrics_10_runs")

In [0]:
metrics_spark_df = spark.table("my_catalog.default.stacking_final_metrics_10_runs")
display(metrics_spark_df)

In [0]:
factors = [10, 50, 100]  # fatores a testar
n_runs = 10  # 10 execuções por fator

original_count = len(test_meta_features)

for factor in factors:
    print(f"\n--- Testando Stacking com fator {factor} ---")
    bigdata_test = pd.concat([test_meta_features] * factor, ignore_index=True)
    bigdata_y_test = np.tile(y_test, factor)

    inference_times_big = []
    memory_usages_big = []
    cpu_percents_big = []
    num_threads_list_big = []

    for _ in range(n_runs):
        start_time = time.time()
        _ = final_meta_learner.predict(bigdata_test)
        inference_time = time.time() - start_time
        inference_times_big.append(inference_time)

        process = psutil.Process(os.getpid())
        memory_usages_big.append(process.memory_info().rss / (1024 * 1024))
        cpu_percents_big.append(psutil.cpu_percent(interval=0.1))
        num_threads_list_big.append(process.num_threads())

    metrics_big = {
        "model": f"StackingEnsemble_BigData_Factor{factor}",
        "inference_time_sec": np.mean(inference_times_big),
        "memory_usage_mb": np.mean(memory_usages_big),
        "cpu_percent": np.mean(cpu_percents_big),
        "num_threads": np.mean(num_threads_list_big),
        "factor": factor,
        "total_records": original_count * factor
    }

    mlflow.log_metric(f"stacking_inference_time_sec_bigdata_factor{factor}", metrics_big["inference_time_sec"])
    mlflow.log_metric(f"stacking_memory_usage_mb_bigdata_factor{factor}", metrics_big["memory_usage_mb"])
    mlflow.log_metric(f"stacking_cpu_percent_bigdata_factor{factor}", metrics_big["cpu_percent"])
    mlflow.log_metric(f"stacking_num_threads_bigdata_factor{factor}", metrics_big["num_threads"])

    efficiency_df_bigdata = pd.DataFrame([metrics_big])
    display(efficiency_df_bigdata)

In [0]:
from pyspark.sql import Row
from pyspark.sql.types import (
    StructType, StructField, StringType, DoubleType, IntegerType
)

schema = StructType([
    StructField("model", StringType(), True),
    StructField("inference_time_sec", DoubleType(), True),
    StructField("memory_usage_mb", DoubleType(), True),
    StructField("cpu_percent", DoubleType(), True),
    StructField("num_threads", DoubleType(), True),
    StructField("factor", IntegerType(), True),
    StructField("total_records", IntegerType(), True)
])

for factor in factors:
    metrics_row_big = Row(
        model=f"StackingEnsemble_BigData_Factor{factor}",
        inference_time_sec=float(np.mean(inference_times_big)),
        memory_usage_mb=float(np.mean(memory_usages_big)),
        cpu_percent=float(np.mean(cpu_percents_big)),
        num_threads=float(np.mean(num_threads_list_big)),
        factor=int(factor),
        total_records=int(original_count * factor)
    )
    metrics_spark_df_big = spark.createDataFrame([metrics_row_big], schema=schema)
    metrics_spark_df_big.write.mode("append").saveAsTable(
        "my_catalog.default.stacking_bigdata_metrics"
    )

In [0]:
metrics_bigdata_df = spark.table("my_catalog.default.stacking_bigdata_metrics")
display(metrics_bigdata_df)